In [1]:
import json
from watson_developer_cloud import AlchemyLanguageV1

alchemy_language = AlchemyLanguageV1(api_key='cb738b85c0e2d0094894bcfe8c73d12d73543c35')

In [2]:
def targeted_document_analysis(url_input, target_words):
    print(json.dumps(
      alchemy_language.targeted_sentiment(
        url=url_input,
        targets=target_words),
      indent=2))
    pass

In [23]:
def document_sentiment(url_input):
    return(json.dumps(
  alchemy_language.sentiment(
    url=url_input),
  indent=2))

In [19]:
def targeted_sentence_analysis(text_input, target_words):
    print(json.dumps(
      alchemy_language.targeted_sentiment(
        text=text_input,
        targets=target_words),
      indent=2))
    pass

In [20]:
def sentence_sentiment(text_input):
    print(json.dumps(
  alchemy_language.sentiment(
    text=text_input),
  indent=2))

In [24]:
document_sentiment("http://docs.oracle.com/cd/E64107_01/bigData.Doc/data_processing_bdd/src/rdp_de_sentiment_svm.html")

'{\n  "status": "OK",\n  "usage": "By accessing AlchemyAPI or using information generated by AlchemyAPI, you are agreeing to be bound by the AlchemyAPI Terms of Use: http://www.alchemyapi.com/company/terms.html",\n  "url": "http://docs.oracle.com/cd/E64107_01/bigData.Doc/data_processing_bdd/src/rdp_de_sentiment_svm.html",\n  "totalTransactions": "1",\n  "language": "english",\n  "docSentiment": {\n    "mixed": "1",\n    "score": "0.496529",\n    "type": "positive"\n  }\n}'